In [1]:
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.layers import Input, Embedding, GRU, LSTM, MaxPooling1D, GlobalMaxPool1D
from keras.layers import Dropout, Dense, Activation, Flatten,Conv1D, SpatialDropout1D
from keras.models import Sequential
from keras.optimizers import RMSprop 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [0]:
max_len = 126
max_words = 10000

In [0]:
df = pd.read_csv('/content/drive/My Drive/MyCovid/abstract_clusters.csv',encoding='utf-8',error_bad_lines=False,engine='python')

In [0]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [0]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
!unzip "/content/drive/My Drive/Paragram/glove.840B.300d.zip"

Archive:  /content/drive/My Drive/Paragram/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [0]:
X =df['Abstract'].values

In [0]:
y =df['Label'].values

In [9]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
    # convert texts to sequences
sequences = tokenizer.texts_to_sequences(X)
    # generate work index
word_index = tokenizer.word_index
    # print top words count
print('{} of unique tokens found'.format(len(word_index)))
    # pad sequences using max_len param
X = pad_sequences(sequences, maxlen=max_len)
    # convert list of labels into numpy array
# labels = np.asarray(labels)
    # print shape of text and label tensors
# print('data tensor shape: {}\nlabel tensor shape:{}'.format(data.shape, labels.shape))

123673 of unique tokens found


In [0]:
embeddings_index = {}
        # open embeddings file
try:
    f = open('glove.840B.300d.txt')
            # iterate over lines and split on individual words
            # split coefficient of word values
            # map words and coefficients to embeddings dictionary
    for line in f:
        values = line.split(' ') # returns list of [word, coeff]
        word = values[0] # gets first list element
        coeff = np.asarray(values[1:], dtype='float32')  # slice coefficiennt value array from remainder of list
                # assign mapping to dictionary
        embeddings_index[word] = coeff
    f.close()
except IOError:
    print('cannot read file. check file paths')

In [0]:
       # prepare glove word-embedding matrix
        # create empty embedding tensor
embedding_matrix = np.zeros((max_words,300))
        # map the top words of the data into the glove embedding matrix
        # words not found from the data in glove will be zeroed
for word, i in word_index.items():
        if i >= max_words: continue
        embedding_vector = embeddings_index.get(word)
        #ALLmight
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector

In [0]:
X_train, X_val_test, y_train, y_val_test = train_test_split(
        X,y, test_size=0.2, random_state=0, stratify = y
        )

In [0]:
X_val, X_test, y_val, y_test = train_test_split(
        X_val_test,y_val_test, test_size=0.5, random_state=0, stratify = y_val_test
        )

In [14]:
model = Sequential()
model.add(Embedding(max_words, 300, weights=[embedding_matrix], trainable=True))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(4))
model.add(GRU(64, dropout=0.1)) 
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          57664     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 3,083,082
Trainable params: 3,083,082
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_train

array([[  43,   45,  359, ...,  202, 2273,  533],
       [   4, 7713,   12, ...,  686,  276, 1004],
       [  10,  488,   13, ..., 1769,  268,  892],
       ...,
       [3296,  828,    1, ...,   13,    1, 7534],
       [ 524,  463,  220, ..., 1804, 1315, 1308],
       [   6,  316,  229, ..., 2013, 2451,  119]], dtype=int32)

In [0]:
Adam = keras.optimizers.Adam(lr = 0.0005)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam, metrics=['sparse_categorical_accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 31238 samples, validate on 3905 samples
Epoch 1/5
31238/31238 [==============================] - 82s 3ms/step - loss: 1.5928 - sparse_categorical_accuracy: 0.4293 - val_loss: 1.4069 - val_sparse_categorical_accuracy: 0.4983
Epoch 2/5
31238/31238 [==============================] - 74s 2ms/step - loss: 1.3312 - sparse_categorical_accuracy: 0.5276 - val_loss: 1.3569 - val_sparse_categorical_accuracy: 0.5170
Epoch 3/5
31238/31238 [==============================] - 72s 2ms/step - loss: 1.2127 - sparse_categorical_accuracy: 0.5691 - val_loss: 1.3513 - val_sparse_categorical_accuracy: 0.5196
Epoch 4/5
31238/31238 [==============================] - 72s 2ms/step - loss: 1.0970 - sparse_categorical_accuracy: 0.6122 - val_loss: 1.3628 - val_sparse_categorical_accuracy: 0.5160
Epoch 5/5
31238/31238 [==============================] - 71s 2ms/step - loss: 0.9614 - sparse_categorical_accuracy: 0.6640 - val_loss: 1.4196 - val_sparse_categorical_accuracy: 0.5104


In [17]:
prediction = model.predict(X_test, verbose = 1)

3905/3905 [==============================] - 1s 226us/step


In [0]:
preds = np.argmax(prediction, axis =1)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.65      0.75      0.69       583
           1       0.45      0.53      0.49       449
           2       1.00      1.00      1.00        34
           3       0.51      0.75      0.61       466
           4       0.46      0.35      0.40       349
           5       0.47      0.37      0.41       482
           6       0.28      0.21      0.24       429
           7       0.61      0.59      0.60       321
           8       0.60      0.65      0.62       405
           9       0.65      0.46      0.54       387

    accuracy                           0.53      3905
   macro avg       0.57      0.57      0.56      3905
weighted avg       0.52      0.53      0.52      3905



In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

0.5316261203585148

In [0]:
# define plotting metrics
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
# plot model training and validation accuracy and loss
plot_training_and_validation(acc, val_acc, loss, val_loss)